# VCPI Group assignment

This notebook aims to get the best possible test accuracy on GTSRB - German Traffic Sign Recognition Benchmark :
It will use CNN for the task.

In this notebook we approach the two-stage data-augmentation first training with original and then together with the data generated

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, model_from_json
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint,TensorBoard, EarlyStopping
from tensorflow.keras.layers import LeakyReLU, BatchNormalization, Conv2D, MaxPooling2D, \
            Dense, Activation, Flatten, Dropout, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import pathlib
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.image import imread
from PIL import Image
import IPython.display as display

import seaborn as sn
import pandas as pd
import random
import glob
tf.config.run_functions_eagerly(True)
gpus = tf.config.list_physical_devices('GPU')
print(tf.__version__)
if gpus:
    print(gpus)
else:
    print("No GPU available.")


BATCH_SIZE = 32
IMAGE_HEIGHT = 32
IMAGE_WIDTH = 32

2.10.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [7]:
data_dir = 'D:/Data-UM/DataVCPI'
train_path = 'D:/Data-UM/DataVCPI/train/GTSRB/Final_Training/Images'
test_path = 'D:/Data-UM/DataVCPI/test'
log_and_model_path = 'logs/model2'

In [3]:
train_data_dir=pathlib.Path(train_path)
test_data_dir=pathlib.Path(test_path)

In [5]:
#count the numbers of images in your directory
train_image_count = len(list(train_data_dir.glob('*/*.png')))
test_image_count = len(list(test_data_dir.glob('*/*.png')))

print('Total image train dataset:', train_image_count)
print('Total image test dataset:',test_image_count)   

#Num of categories:
NUM_CLASSES = len(os.listdir(train_path))
print('Number of classes/categories: ',NUM_CLASSES)         

Total image train dataset: 39209
Total image test dataset: 12630
Number of classes/categories:  43


In [6]:
# Label Overview
classesNames = { 0:'Speed limit (20km/h)',
            1:'Speed limit (30km/h)', 
            2:'Speed limit (50km/h)', 
            3:'Speed limit (60km/h)', 
            4:'Speed limit (70km/h)', 
            5:'Speed limit (80km/h)', 
            6:'End of speed limit (80km/h)', 
            7:'Speed limit (100km/h)', 
            8:'Speed limit (120km/h)', 
            9:'No passing', 
            10:'No passing veh over 3.5 tons', 
            11:'Right-of-way at intersection', 
            12:'Priority road', 
            13:'Yield', 
            14:'Stop', 
            15:'No vehicles', 
            16:'Veh > 3.5 tons prohibited', 
            17:'No entry', 
            18:'General caution', 
            19:'Dangerous curve left', 
            20:'Dangerous curve right', 
            21:'Double curve', 
            22:'Bumpy road', 
            23:'Slippery road', 
            24:'Road narrows on the right', 
            25:'Road work', 
            26:'Traffic signals', 
            27:'Pedestrians', 
            28:'Children crossing', 
            29:'Bicycles crossing', 
            30:'Beware of ice/snow',
            31:'Wild animals crossing', 
            32:'End speed + passing limits', 
            33:'Turn right ahead', 
            34:'Turn left ahead', 
            35:'Ahead only', 
            36:'Go straight or right', 
            37:'Go straight or left', 
            38:'Keep right', 
            39:'Keep left', 
            40:'Roundabout mandatory', 
            41:'End of no passing', 
            42:'End no passing veh > 3.5 tons' }

classes = np.array([
    '00000', '00001', '00002', '00003', '00004', '00005', '00006', '00007',
    '00008', '00009', '00010', '00011', '00012', '00013', '00014', '00015',
    '00016', '00017', '00018', '00019', '00020', '00021', '00022', '00023',
    '00024', '00025', '00026', '00027', '00028', '00029', '00030', '00031',
    '00032', '00033', '00034', '00035', '00036', '00037', '00038', '00039',
    '00040', '00041', '00042'
])